**To scape product information, changing vpn may not be so convenient to prevent blocking. So here we try to use another methods: changing the proxies.**

# Preparation: Import packages

In [1]:
import requests
import re
import random
from bs4 import BeautifulSoup
import time
import json
import numpy as np
import pandas as pd

In [2]:
# How many products?
products = open('product_link.txt')
count = 0
for item in products:
    count += 1
print('Product counts: ', count)

Product counts:  3085


### Given free proxies always have problems, we now just use workable proxies which have been checked by hand.
* Here we get these free proxies from [here](http://www.freeproxylists.net/zh/?c=US&pr=HTTPS&u=80&s=ts)
* We cannot simply scape the free proxies on this website because it is writen in JavaScipt. We might try to use selenium to scape the links, which would be a future plan to optimize the scaping. So far, we have to copy several links from this website and put them into the proxies list

In [3]:
proxies_lst = ['140.227.52.41:6000', '140.227.167.33:45210', '140.227.175.231:6000', '140.227.202.194:6000',
              '140.227.230.89:60088', '134.209.98.103:8080', '157.245.153.50:8080', '157.245.62.226:8080',
              '157.245.153.58:8080', '205.185.115.100:8080', '157.245.199.136:8080', '68.183.125.165:8080']

In [4]:
def get_data(product_link, proxy=None):
    # try to use the proxy to scape data.
    try:
        response = requests.get(product_link, proxies={
                                "http": proxy, "https": proxy}, timeout=15)
    except:
        return 'Cannot connect to Sephora'

    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    product_id = re.findall(R'P[0-9]{3,6}', product_link)[0]

    # Some product may have been removed.
    if soup.find('h1', class_='css-56434t'):
        return None

    # Get brand name and product name
    for brand_and_name in soup.find_all('h1', class_='css-a1jw00'):
        names = [names.get_text() for names in brand_and_name.find_all('span')]
        brand = names[0]
        prd_name = names[1]

    # Get Category and price (use json to get categories and price),
    # Because it would give us sub categories and offer price.
    # But if that doesn't work, just get main category and price box.
    dic_json = soup.find_all(attrs={"type": "application/ld+json"})
    try:
        json_category = json.loads(dic_json[0].get_text())
        price = json.loads(dic_json[1].get_text())['offers'][0]['price']
        category_lst = [item['item']['name']
                        for item in json_category['itemListElement']]
        category = '/'.join(category_lst)

    except:
        try:
            category = [cate.get_text()
                        for cate in soup.find_all('ol', class_='css-1doqpel')]
            price = soup.find(
                'div', attrs={"data-comp": "Price Box"}).get_text()
        except:
            price = 'na'
            category = 'na'

    # Get item number and size
    size_and_item = soup.find(attrs={"data-comp": "SizeAndItemNumber Box"})
    # If there's only item number and no size info, it will be out of index
    try:
        item_num = size_and_item.contents[1].split(' ')[1]
    except:
        item_num = size_and_item.contents[0].split(' ')[1]
    # If there's no size at SizeAndItemNumber Box, then try use description area to see.
    try:
        size = size_and_item.span.contents[0].split('SIZE ')[1]
    except:
        try:
            size = soup.find(
                "span", attrs={"class": "css-12wl10d"}).contents[-1]
        except:
            size = 'na'

    # Get love counts
    try:
        love_counts = soup.find(
            'span', attrs={"data-at": "product_love_count"}).get_text()
    except:
        love_counts = 'na'

    # review nums and ratings
    link_json = soup.find(attrs={"id": "linkJSON"})
    json_str = str(link_json)
    ratings = re.findall(R'\"rating\"\:(.*?)\,', json_str)
    reviews = re.findall(R'\"reviews\"\:(.*?)\,', json_str)
    try:
        rating = ratings[0]
    except:
        rating = 'na'
    try:
        reviews_count = reviews[0]
    except:
        reviews_count = 'na'

    dic1 = {}
    dic1['Product_Id'] = product_id
    dic1['product_name'] = prd_name
    dic1['item_num'] = item_num
    dic1['brand'] = brand
    dic1['category'] = category
    dic1['price'] = price
    dic1['size'] = size
    dic1['love_count'] = love_counts
    dic1['rating'] = rating
    dic1['reviews_count'] = reviews_count
    dic1['link'] = product_link
    return dic1

In [5]:
# Given a proxies list, find the proxy that can work and return that proxy for later scaping.
def find_good_proxy(link, proxies_lst):
    """Given a proxies list, find the proxy that 
    can work and return that proxy for later scaping."""
    # Shuffle the list
    random.shuffle(proxies_lst)
    for proxy_ in proxies_lst:
        product_data = get_data(link, proxy=proxy_)
        # If that proxy cannot work, continue to test the next proxy.
        if product_data == 'Cannot connect to Sephora':
            continue
        # If that proxy works, so just return the proxy and the product info
        elif product_data:
            print('using good proxy: ', proxy_)
            return proxy_, product_data
        # If the proxy also works, but the product information is none, then still return them.
        else:
            return proxy_, None

In [6]:
def scaping_data(start=0, end=None):
    """Scape product data"""
    # Get the list of product links
    products = open('product_link.txt')
    product_links = [item.rstrip() for item in products]
    frame = []

    # First time to use the first proxy to do scaping
    proxy_ = proxies_lst[0]

    # Scape data from each product link
    for link in product_links[start:end]:
        print(start, link)

        # try to scape data with the previous saved proxy
        product_data = get_data(link, proxy_)

        # If the proxy cannot work, then try to find the next workable proxy
        if product_data == 'Cannot connect to Sephora':
            try:
                proxy_, product_data = find_good_proxy(link, proxies_lst)
                df = pd.DataFrame(product_data, index=[start])
                frame.append(df)
                start += 1
            # If all proxies cannot work
            except:
                return pd.concat(frame)
        # If the proxy can work
        elif product_data:
            df = pd.DataFrame(product_data, index=[start])
            frame.append(df)
            start += 1

    return pd.concat(frame)

In [7]:
res = scaping_data()
res

0 https://www.sephora.com/product/blu-mediterraneo-minature-set-P443401?icid2=products
1 https://www.sephora.com/product/colonia-P163604?icid2=products
using good proxy:  157.245.153.50:8080
2 https://www.sephora.com/product/rosa-nobile-P388670?icid2=products
3 https://www.sephora.com/product/blu-mediterraneo-fico-di-amalfi-P307801?icid2=products
4 https://www.sephora.com/product/blu-mediterraneo-mirto-di-panarea-P307804?icid2=products
5 https://www.sephora.com/product/colonia-miniature-set-P443400?icid2=products
6 https://www.sephora.com/product/colonia-pura-P422289?icid2=products
7 https://www.sephora.com/product/le-nobili-collection-set-P431439?icid2=products
8 https://www.sephora.com/product/colonia-essenza-P269110?icid2=products
9 https://www.sephora.com/product/colonia-intensa-oud-P377678?icid2=products
10 https://www.sephora.com/product/blu-mediterraneo-arancia-di-capri-gift-set-P450817?icid2=products
11 https://www.sephora.com/product/blu-mediterraneo-mandorlo-di-sicilia-P30780

93 https://www.sephora.com/product/mini-loose-highlighter-set-P451598?icid2=products
94 https://www.sephora.com/product/best-brows-ever-kit-P451601?skuId=2289502&icid2=products
95 https://www.sephora.com/product/brow-gel-P69304?icid2=products
96 https://www.sephora.com/product/undressed-lip-set-P450064?icid2=products
97 https://www.sephora.com/product/norvina-pro-pigment-palette-vol-3-P450846?icid2=products
98 https://www.sephora.com/product/jackie-aina-eyeshadow-palette-P449147?icid2=products
99 https://www.sephora.com/product/stick-foundation-P410504?icid2=products
100 https://www.sephora.com/product/invigorating-rush-hair-body-wash-P376886?icid2=products
101 https://www.sephora.com/product/ingrown-hair-treatment-P170545?icid2=products
102 https://www.sephora.com/product/facial-scrub-P9036?icid2=products
103 https://www.sephora.com/product/glycolic-exfoliating-resurfacing-wipes-P397373?icid2=products
104 https://www.sephora.com/product/astringent-toner-pads-P189131?icid2=products
105

186 https://www.sephora.com/product/barepro-performance-wear-powder-foundation-P410543?icid2=products
187 https://www.sephora.com/product/bareminerals-matte-foundation-broad-spectrum-spf-15-P240604?icid2=products
188 https://www.sephora.com/product/mineral-veil-P61004?icid2=products
189 https://www.sephora.com/product/barepro-performance-wear-liquid-foundation-broad-spectrum-spf-20-P421001?icid2=products
190 https://www.sephora.com/product/barepro-16-hr-full-coverage-concealer-P428606?icid2=products
191 https://www.sephora.com/product/bareskin-reg-complete-coverage-serum-concealer-P399811?icid2=products
192 https://www.sephora.com/product/complexion-rescue-hydrating-foundation-stick-broad-spectrum-spf-25-P441829?icid2=products
193 https://www.sephora.com/product/bareminerals-nothing-beats-original-4-piece-get-started-kit-P421000?icid2=products
194 https://www.sephora.com/product/blemish-rescue-skin-clearing-loose-powder-foundation-P432867?icid2=products
195 https://www.sephora.com/prod

277 https://www.sephora.com/product/7-dwarfs-mini-lipstick-set-P427704?icid2=products
278 https://www.sephora.com/product/boudoir-short-hair-contour-brush-P396821?icid2=products
279 https://www.sephora.com/product/hair-redemption-butter-masque-P451206?icid2=products
280 https://www.sephora.com/product/lift-me-up-hair-thickener-P451204?icid2=products
281 https://www.sephora.com/product/volume-restore-kit-P451200?icid2=products
282 https://www.sephora.com/product/wake-up-call-volumizing-shampoo-P451210?icid2=products
283 https://www.sephora.com/product/second-chance-repairing-shampoo-P451208?icid2=products
284 https://www.sephora.com/product/second-chance-repairing-conditioner-P451207?icid2=products
285 https://www.sephora.com/product/scalp-detox-hair-volumizing-kit-P451199?icid2=products
286 https://www.sephora.com/product/wake-up-call-volumizing-conditioner-P451209?icid2=products
287 https://www.sephora.com/product/silver-lining-purple-butter-masque-P451205?icid2=products
288 https://w

366 https://www.sephora.com/product/rosarco-blow-dry-perfection-heat-protectant-creme-P411359?icid2=products
367 https://www.sephora.com/product/scalp-revival-tm-scalp-soothing-travel-kit-P440959?icid2=products
368 https://www.sephora.com/product/blossom-bloom-ginseng-biotin-volumizing-shampoo-P402071?icid2=products
369 https://www.sephora.com/product/refresh-restart-set-P450070?icid2=products
370 https://www.sephora.com/product/hairdresser-s-invisible-oil-primer-P386464?icid2=products
371 https://www.sephora.com/product/reign-dear-texture-set-P449780?icid2=products
372 https://www.sephora.com/product/thickening-volume-shampoo-P280566?icid2=products
373 https://www.sephora.com/product/hairdresser-s-invisible-oil-conditioner-P386462?icid2=products
374 https://www.sephora.com/product/thickening-dryspun-finish-P381033?icid2=products
375 https://www.sephora.com/product/hairdresser-s-invisible-oil-P375391?icid2=products
376 https://www.sephora.com/product/hairdresser-s-invisible-oil-shampoo

...

2752 https://www.sephora.com/product/amazonian-clay-12-hour-full-coverage-foundation-spf-15-P302918?icid2=products
2753 https://www.sephora.com/product/sea-limited-edition-drink-of-h2o-hydrating-boost-P450597?icid2=products
2754 https://www.sephora.com/product/chrome-for-holidays-set-P450532?icid2=products
2755 https://www.sephora.com/product/sea-quench-lip-rescue-balm-P405601?icid2=products
2756 https://www.sephora.com/product/lights-camera-lashes-P111902?icid2=products
2757 https://www.sephora.com/product/bb-tinted-treatment-12-hour-primer-broad-spectrum-spf-30-sunscreen-P377542?icid2=products
2758 https://www.sephora.com/product/tarteist-party-3-liner-trio-P450533?icid2=products
2759 https://www.sephora.com/product/taste-beauty-succulent-lip-balm-P452693?icid2=products
2760 https://www.sephora.com/product/regenerating-cleanser-P379710?icid2=products
2761 https://www.sephora.com/product/resurfacing-mask-P379707?icid2=products
2762 https://www.sephora.com/product/tatas-daily-essential

2844 https://www.sephora.com/product/just-like-heaven-rollerball-P429422?icid2=products
2845 https://www.sephora.com/product/tory-burch-bel-azur-rollerball-P420535?icid2=products
2846 https://www.sephora.com/product/tory-burch-nuit-azur-rollerball-P432803?icid2=products
2847 https://www.sephora.com/product/metallist-sparkling-foiled-eye-shadow-P441017?icid2=products
2848 https://www.sephora.com/product/metalist-liquid-foil-finish-shadow-P409990?skuId=2041572&icid2=products
2849 https://www.sephora.com/product/pretty-filter-mattish-liquid-eyeshadow-P447392?icid2=products
2850 https://www.sephora.com/product/no-poreblem-primer-P419640?icid2=products
2851 https://www.sephora.com/product/metallist-high-shine-bouncy-cream-shadow-palette-P429023?icid2=products
2852 https://www.sephora.com/product/beachplease-tinted-balm-blush-P449342?icid2=products
2853 https://www.sephora.com/product/shineon-lip-jelly-P448854?icid2=products
2854 https://www.sephora.com/product/sos-save-our-skin-daily-rescue

2940 https://www.sephora.com/product/flowerbomb-midnight-P441723?icid2=products
2941 https://www.sephora.com/product/fun-sized-mini-eyeshadow-palette-P437714?icid2=products
2942 https://www.sephora.com/product/vibes-neon-pressed-pigments-P449088?icid2=products
2943 https://www.sephora.com/product/violet-voss-oh-snap-gingerbread-fun-sized-mini-eyeshadow-palette-P452276?icid2=products
2944 https://www.sephora.com/product/sparkling-cherry-limeade-palette-P451645?icid2=products
2945 https://www.sephora.com/product/pro-eyeshadow-palette-hg-P425705?icid2=products
2946 https://www.sephora.com/product/best-life-2-palette-P451922?icid2=products
2947 https://www.sephora.com/product/coral-crush-palette-P448850?icid2=products
2948 https://www.sephora.com/product/best-life-eyeshadow-palette-P446020?icid2=products
2949 https://www.sephora.com/product/pro-eyeshadow-palette-ride-or-die-P425707?icid2=products
2950 https://www.sephora.com/product/sugar-crystals-eyeshadow-palette-P442876?icid2=products
2

3032 https://www.sephora.com/product/nude-illusion-liquid-foundation-P437367?icid2=products
3033 https://www.sephora.com/product/rise-prime-balm-primer-duo-P417144?icid2=products
3034 https://www.sephora.com/product/prep-set-essentials-kit-P451646?icid2=products
3035 https://www.sephora.com/product/the-super-elixir-refill-P437503?icid2=products
3036 https://www.sephora.com/product/sleep-welle-fortified-calming-tea-refill-P437510?icid2=products
3037 https://www.sephora.com/product/sleep-welle-fortified-calming-tea-P437509?icid2=products
3038 https://www.sephora.com/product/glimmer-balm-P436015?icid2=products
3039 https://www.sephora.com/product/mini-lip-pill-kit-P436016?icid2=products
3040 https://www.sephora.com/product/matcha-lip-balm-P436014?icid2=products
3041 https://www.sephora.com/product/pug-pout-lip-balm-kit-P436017?icid2=products
3042 https://www.sephora.com/product/purrfect-pout-lipstick-P440525?icid2=products
3043 https://www.sephora.com/product/superberry-hydrate-glow-dream

,Product_Id,product_name,item_num,brand,category,price,size,love_count,rating,reviews_count,link
0,P443401,Blu Mediterraneo MINIATURE Set,2218774,Acqua Di Parma,Fragrance/Value & Gift Sets/Perfume Gift Sets,63.0,5 x 0.16oz/5mL,2603,4,4,https://www.sephora.com/product/blu-mediterran...
1,P163604,Colonia,2044816,Acqua Di Parma,Fragrance/Men/Cologne,63.0,0.7 oz/ 20 mL,2537,4.3067,75,https://www.sephora.com/product/colonia-P16360...
2,P388670,Rosa Nobile,1638832,Acqua Di Parma,Fragrance/Women/Perfume,68.0,3.4 oz/ 101 mL,4582,4.5652,69,https://www.sephora.com/product/rosa-nobile-P3...
3,P307801,Fico di Amalfi,1417609,Acqua Di Parma,Fragrance/Women/Perfume,64.0,5 oz/ 148 mL,2472,4.5714,70,https://www.sephora.com/product/blu-mediterran...
4,P307804,Mirto di Panarea,1417617,Acqua Di Parma,Fragrance/Women/Perfume,64.0,2.5 oz/ 74 mL,2624,4.6364,22,https://www.sephora.com/product/blu-mediterran...
...,...,...,...,...,...,...,...,...,...,...,...
3078,P428707,Orange Blossom Eau de Parfum Rollerball,2047728,The 7 Virtues,Fragrance/Women/Rollerballs & Travel Size,29.0,0.33 oz/ 11 mL,2009,4.2083,24,https://www.sephora.com/product/orange-blossom...
3079,P428716,Vetiver Elemi Eau de Parfum Rollerball,2047702,The 7 Virtues,Fragrance/Women/Rollerballs & Travel Size,29.0,0.33 oz/ 11 mL,1230,4.6,15,https://www.sephora.com/product/vetiver-elemi-...
3080,P428709,Patchouli Citrus Eau de Parfum Rollerball,2047686,The 7 Virtues,Fragrance/Women/Rollerballs & Travel Size,29.0,0.33 oz/ 11 mL,1336,4.3077,26,https://www.sephora.com/product/patchouli-citr...
3081,P452918,8Greens Gummies Dietary Supplement,2298891,8Greens,Skincare/Wellness/Beauty Supplements,45.0,60 Gummies,1668,4.3333,9,https://www.sephora.com/product/8greens-8green...


In [8]:
res.to_csv('Sephora_product_info.csv', index=False)
res

,Product_Id,product_name,item_num,brand,category,price,size,love_count,rating,reviews_count,link
0,P443401,Blu Mediterraneo MINIATURE Set,2218774,Acqua Di Parma,Fragrance/Value & Gift Sets/Perfume Gift Sets,63.0,5 x 0.16oz/5mL,2603,4,4,https://www.sephora.com/product/blu-mediterran...
1,P163604,Colonia,2044816,Acqua Di Parma,Fragrance/Men/Cologne,63.0,0.7 oz/ 20 mL,2537,4.3067,75,https://www.sephora.com/product/colonia-P16360...
2,P388670,Rosa Nobile,1638832,Acqua Di Parma,Fragrance/Women/Perfume,68.0,3.4 oz/ 101 mL,4582,4.5652,69,https://www.sephora.com/product/rosa-nobile-P3...
3,P307801,Fico di Amalfi,1417609,Acqua Di Parma,Fragrance/Women/Perfume,64.0,5 oz/ 148 mL,2472,4.5714,70,https://www.sephora.com/product/blu-mediterran...
4,P307804,Mirto di Panarea,1417617,Acqua Di Parma,Fragrance/Women/Perfume,64.0,2.5 oz/ 74 mL,2624,4.6364,22,https://www.sephora.com/product/blu-mediterran...
...,...,...,...,...,...,...,...,...,...,...,...
3078,P428707,Orange Blossom Eau de Parfum Rollerball,2047728,The 7 Virtues,Fragrance/Women/Rollerballs & Travel Size,29.0,0.33 oz/ 11 mL,2009,4.2083,24,https://www.sephora.com/product/orange-blossom...
3079,P428716,Vetiver Elemi Eau de Parfum Rollerball,2047702,The 7 Virtues,Fragrance/Women/Rollerballs & Travel Size,29.0,0.33 oz/ 11 mL,1230,4.6,15,https://www.sephora.com/product/vetiver-elemi-...
3080,P428709,Patchouli Citrus Eau de Parfum Rollerball,2047686,The 7 Virtues,Fragrance/Women/Rollerballs & Travel Size,29.0,0.33 oz/ 11 mL,1336,4.3077,26,https://www.sephora.com/product/patchouli-citr...
3081,P452918,8Greens Gummies Dietary Supplement,2298891,8Greens,Skincare/Wellness/Beauty Supplements,45.0,60 Gummies,1668,4.3333,9,https://www.sephora.com/product/8greens-8green...


## To do: try to scape the free proxies by using function

In [ ]:
def get_proxies():
    url = 'https://www.sslproxies.org/'
    response = requests.get(url)
    html = response.text
    tables = pd.read_html(html)
    proxies_table = tables[0].dropna()
    proxies_table_c = proxies_table.copy()
    proxies_table_c.loc[~proxies_table_c['Port'].isna(), 'Port'] = proxies_table['Port'].astype('int64')
    proxies_table_new = proxies_table_c.loc[proxies_table['Https']=='yes'].reset_index().iloc[:10,]
    proxies_table_new['proxies'] = (proxies_table_new['IP Address'] + ':' + proxies_table_new['Port'].astype('str'))
    proxies_lst = proxies_table_new['proxies'].to_list()
    print('get proxies list:', proxies_lst)
    return proxies_lst

In [ ]:
def try_proxy(link, proxies_lst,i=0):
    print('trying to find workable proxy')
    try:
        work_proxy = proxies_lst[i]
        print('proxy:', work_proxy )
        product_data = get_data(link, proxies={"http": work_proxy, "https": work_proxy})
        print('Using proxy: '+ work_proxy)
        return work_proxy, product_data
    except:
        print(work_proxy + ' not working')
        i+=1
        try_proxy(link, proxies_lst, i=i)

In [ ]:
def get_data_w_proxy(link):
    try:
        proxies_lst = get_proxies()
        print('Get a new proxies_lst')
        work_proxy = try_proxy(product_links[0], proxies_lst)
        print('Get a new workable proxy')
        product_data = get_data(link, proxies={"http": work_proxy, "https": work_proxy})
        print('Successfully get data')
        return product_data
    except:
        print('The proxies cannot work, and trying to find more proxies')
        get_data_w_proxy(link)

In [ ]:
def scaping_data(start=0,end=None):
    products = open('product_link.txt')
    product_links = [item.rstrip() for item in products]
    frame = []
    for link in product_links[start:end]: 
        print(start, link)
        product_data = get_data_w_proxy(link)
        if product_data:
            df = pd.DataFrame(product_data,index=[start])
            frame.append(df)
            start+=1
    return pd.concat(frame)